# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

### Описание данных

**Признаки**

- *DateCrawled* — дата скачивания анкеты из базы
- *VehicleType* — тип автомобильного кузова
- *RegistrationYear* — год регистрации автомобиля
- *Gearbox* — тип коробки передач
- *Power* — мощность (л. с.)
- *Model* — модель автомобиля
- *Kilometer* — пробег (км)
- *RegistrationMonth* — месяц регистрации автомобиля
- *FuelType* — тип топлива
- *Brand* — марка автомобиля
- *NotRepaired* — была машина в ремонте или нет
- *DateCreated* — дата создания анкеты
- *NumberOfPictures* — количество фотографий автомобиля
- *PostalCode* — почтовый индекс владельца анкеты (пользователя)
- *LastSeen* — дата последней активности пользователя

**Целевой признак**

- *Price* — цена (евро)

## Подготовка данных

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)

# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [3]:
df_car = pd.read_csv('/datasets/autos.csv')

#df_car = pd.read_csv('autos.csv')

Изучим данные:

In [4]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df_car.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
df_car.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Подсчитаем количество пропусков:

In [7]:
display(df_car.isna().sum())

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Чтобы определить, как заполнять пропуски и нужно ли их заполнять, посмотрим на каждый столбец подробнее, чтобы оценить его важность как признака, влияющего на стоимость авто:

In [8]:
# Для отображения всех столбцов сделаем цикл, который выводит каждый столобец по очереди 
# (его уникальные значения и их количество)
def columns_info(data_frame):
    for column in data_frame.columns:
        print(column)
        display(data_frame[column].value_counts())  
        display(data_frame[column].unique()) 
        print()

In [1]:
#columns_info(df_car)

1. В столбце с целевым признаком есть значение 0 - навряд ли кто-то будет продавать машину за бесплатно. Также есть значения 1,2,3 евро и 50, что тоже похоже на неправдивую информацию.
2. Дата скачивания анкеты из базы никак не влияет на цену авто.
3. Тип кузова важный признак, но столбец содержит пропуски.
4. В столбце с годом регистрации есть аномалии - 100 год, 9999 год, 5000. Нужно убрать аномалии и просмотреть на автомобили, которые заявлены от 1910 года и 1890.
5. Тип коробки передач также влияет на стоимость авто, но в столбце есть пропуски.
6. В столбце с мощностью авто также еть аномалии - значение 0 встречается чаще всего, также есть значения меньше 10, а также есть значения больше 1000 (даже 17000 тысяч есть - самолет что ли?)
7. Название модели тоже очень важно, но в столбце есть пропуски.
8. Пробег также влияет на цену, пропусков нет, аномалий тоже нет.
9. Месяц регистрации на цену не влияет, столбец не важен.
10. В типе топлива есть пропуски, столбец важен.
11. Марка автомобиля важна, пропусков нет.
12. Есть пропуски в информации о том, была ли машина отремонтирована или нет, информация об этом важна.
13. Дата создания анкеты на цену не влияет.
14. К сожалению, столбец с количеством фото заполнен только 0, интересно, влияет ли на цену этот или нет.
15. Почтовый индекс показывает, где машина была выставлена на продажу, что, теоретически, может влиять на цену (разные регионы - разные условия экспуатации), но 4-х значные коды без указания страны не несут пользы.
16. Дата последней активности пользователя не влияет на цену авто.

Удалим ненужные столбцы, а также аномалии, после заполним пропуски.

### Аномалии

In [9]:
df_car_pure = df_car.drop(['DateCrawled', 
                           'RegistrationMonth', 
                           'DateCreated', 
                           'NumberOfPictures', 
                           'PostalCode', 
                           'LastSeen'], axis='columns')

In [10]:
df_car_pure.query('1960 > RegistrationYear > 1800').head(15)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
15,450,small,1910,NaN,0,ka,5000,petrol,ford,NaN
1928,7000,suv,1945,manual,48,other,150000,petrol,volkswagen,no
2273,1800,convertible,1925,NaN,0,NaN,5000,NaN,sonstige_autos,no
3333,10500,sedan,1955,manual,30,other,60000,petrol,ford,NaN
6629,0,small,1910,NaN,0,NaN,5000,other,sonstige_autos,NaN
6977,135,NaN,1910,NaN,0,NaN,5000,NaN,opel,NaN
10183,1250,NaN,1910,NaN,0,other,5000,NaN,audi,NaN
12992,0,small,1954,manual,54,corsa,150000,petrol,opel,no
12993,11000,other,1955,manual,40,NaN,50000,petrol,sonstige_autos,no
13177,2300,sedan,1958,manual,45,NaN,80000,petrol,sonstige_autos,NaN


Chevrolet Spark 1910 года выглядит совсем неправдоподобно, а вот volkswagen kaefer 1959 уже больше похоже на правду, поэтому уберем те объявления, где год указан до 1950 (также по причине того, что у многих машин, где указаны 1930-1940 не указана модель, чтобы оценить правдивы ли данные). А так как объявления выставлены в 2016, уберем все после 2017.

In [11]:
df_car_pure = df_car_pure.query('2018 > RegistrationYear > 1949')

In [12]:
df_car_pure.query('23 > Power > 20')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
15693,1000,wagon,1999,manual,22,astra,150000,petrol,opel,no
16902,19000,suv,1960,manual,22,NaN,60000,petrol,sonstige_autos,no
19106,1500,small,1960,manual,21,other,5000,petrol,renault,yes
72592,350,small,1990,NaN,21,corsa,5000,petrol,opel,yes
101909,3600,NaN,2017,manual,21,500,20000,petrol,fiat,NaN
162410,2950,sedan,1959,manual,21,other,150000,petrol,renault,yes
184795,1000,wagon,1960,manual,21,other,5000,petrol,trabant,no
219940,4999,sedan,1955,manual,22,other,20000,petrol,opel,no
259500,1450,NaN,1980,manual,21,500,150000,petrol,fiat,yes
269854,9350,NaN,1980,manual,22,500,90000,petrol,fiat,NaN


In [13]:
df_car_pure.query('1000 > Power > 500').head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
3746,0,NaN,2017,manual,750,NaN,150000,petrol,smart,no
4060,3100,sedan,2005,manual,953,colt,150000,gasoline,mitsubishi,no
6296,599,small,2002,manual,603,matiz,5000,petrol,chevrolet,yes
6504,3000,small,2009,manual,771,punto,125000,petrol,fiat,NaN
6637,600,small,1996,manual,603,corsa,150000,petrol,opel,yes
8057,500,small,2002,manual,620,ypsilon,150000,petrol,lancia,NaN
12740,3500,convertible,2003,manual,952,ka,70000,petrol,ford,no
13373,18500,small,2002,auto,600,s_type,150000,NaN,jaguar,no
15433,0,coupe,1995,manual,999,escort,5000,NaN,ford,no
16385,100,small,1996,manual,553,NaN,150000,NaN,renault,yes


Старые автомобили могут иметь меньше 30 л.с, но навряд ли будут слабее 22 л.с. Быть мощнее 500 Мерседес может, а вот форд фиеста с 750 л.с. навряд ли будет стоить 450 евро.

In [14]:
df_car_pure = df_car_pure.query('501 > Power > 22')

In [15]:
df_car_pure.query('300 > Price > 0').head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
59,1,suv,1994,manual,286,NaN,150000,NaN,sonstige_autos,NaN
60,200,coupe,1995,manual,102,3er,150000,petrol,bmw,NaN
79,150,NaN,2016,manual,75,astra,150000,petrol,opel,yes
80,250,wagon,2000,manual,155,156,150000,petrol,alfa_romeo,yes
89,1,sedan,1995,manual,113,e_klasse,150000,gasoline,mercedes_benz,no
92,250,NaN,2000,NaN,60,ka,150000,NaN,ford,NaN
268,1,sedan,1990,manual,90,80,70000,petrol,audi,no
281,150,NaN,2016,auto,75,fiesta,50000,petrol,ford,yes
408,1,NaN,2000,NaN,75,golf,90000,petrol,volkswagen,NaN
452,250,small,1994,manual,102,3er,150000,petrol,bmw,NaN


Так как данные во всей таблице очень странные, уберем в столбце с ценой только явные выбросы (ну не может машина 2016 года стоить 300 евро).

In [16]:
df_car_pure = df_car_pure.query('Price > 350')

In [17]:
df_car_pure.describe()

,Price,RegistrationYear,Power,Kilometer
count,291342.000000,291342.000000,291342.000000,291342.000000
mean,5009.175402,2003.356729,122.045695,128145.221080
std,4588.649579,6.690633,53.316020,36688.109487
min,355.000000,1950.000000,23.000000,5000.000000
25%,1500.000000,1999.000000,80.000000,125000.000000
50%,3390.000000,2004.000000,115.000000,150000.000000
75%,7150.000000,2008.000000,150.000000,150000.000000
max,20000.000000,2017.000000,500.000000,150000.000000


### Пропуски

Заполним пропуски в столбцах с названием модели, типом кузова, типом коробки передач, а также была ли отремонтирована машина:

In [18]:
df_car_pure[df_car_pure['Brand'] == 'smart']['VehicleType'].value_counts()

small          2965
convertible    1058
coupe           294
sedan            27
wagon             9
other             4
Name: VehicleType, dtype: int64

Качество данных оставляет желать лучшего, универсал от Smart это что-то новое...

Название модели нельзя восстановить, брать самое популярное значение будет странно (может появится Chevrolet Spark 1950 года), поэтому заполним значением 'unknown'. 

Также можно поступить и с типом кузова (универсалы от Smart у нас уже есть). 


Скорее всего пропуски в столбце с ремонтом автомобиля связаны с тем, что ремонт не проводился, поэтому этот столбец не был заполнен, заполним словом 'yes'.

Пропуск с типом коробки передач - 0,018% данных - заполним типом 'manual', который встречается чаще.

In [19]:
df_car_pure['NotRepaired'].fillna(value='yes', inplace=True)

In [20]:
df_car_pure['Model'].fillna(value='unknown', inplace=True)

In [21]:
df_car_pure['VehicleType'].fillna(value='unknown', inplace=True)

In [22]:
df_car_pure['Gearbox'].fillna(value='manual', inplace=True)

In [23]:
df_car_pure['FuelType'].value_counts()

petrol      181094
gasoline     88450
lpg           4695
cng            484
hybrid         203
other           76
electric        36
Name: FuelType, dtype: int64

Тип топлива 92,5% это бензин, но поробуем восстаноить тип в зависимости от модели и посмотрим, что получится:

In [24]:
df_car_pure['FuelType'] = (df_car_pure['FuelType']
                           .fillna(df_car_pure.groupby('Model')['FuelType']
                           .transform(lambda x: x.value_counts().idxmax())))

In [25]:
df_car_pure['FuelType'].value_counts()

petrol      194392
gasoline     91456
lpg           4695
cng            484
hybrid         203
other           76
electric        36
Name: FuelType, dtype: int64

Часть пропусков заполнилась 'petrol', часть 'gasoline', в приницпе можно было так сделать и методом fillna.

Проверим отсутствие пропусков:

In [26]:
display(df_car_pure.isna().sum())

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
NotRepaired         0
dtype: int64

In [27]:
df_car_pure.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
1,18300,coupe,2011,manual,190,unknown,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,yes
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes


Приведем столбцы `'Gearbox'` и `'NotRepaired'` к бинарному типу:

* В столбце `'Gearbox'`: 0 - manual, 1 - auto 	
* В столбце `'NotRepaired'`: 0 - no, 1 - yes

И заменим название столбца `'Gearbox'` на `'AutoGearbox'`

In [28]:
df_car_pure['Gearbox'] = df_car_pure['Gearbox'].replace('manual', 0)
df_car_pure['Gearbox'] = df_car_pure['Gearbox'].replace('auto', 1)

df_car_pure['NotRepaired'] = df_car_pure['Gearbox'].replace('no', 0)
df_car_pure['NotRepaired'] = df_car_pure['Gearbox'].replace('yes', 1)

In [29]:
df_car_pure.columns = (['Price', 'VehicleType', 'RegistrationYear', 'AutoGearbox', 'Power', 'Model',
       'Kilometer', 'FuelType', 'Brand', 'NotRepaired'])

In [30]:
df_car_pure.sample(10)

,Price,VehicleType,RegistrationYear,AutoGearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
25414,3100,sedan,2005,0,75,golf,150000,petrol,volkswagen,0
223882,1250,sedan,1994,0,122,c_klasse,150000,petrol,mercedes_benz,0
341776,4950,bus,2005,0,116,touran,100000,petrol,volkswagen,0
87210,2800,sedan,1989,1,170,5er,100000,gasoline,bmw,1
122203,600,sedan,1996,0,60,golf,150000,petrol,volkswagen,0
73957,700,sedan,1995,0,60,cordoba,150000,petrol,seat,0
68529,5700,small,2008,0,75,500,125000,gasoline,fiat,0
73504,550,wagon,1998,0,101,3er,150000,petrol,bmw,0
115107,1790,wagon,2004,0,140,156,150000,lpg,alfa_romeo,0
213129,9450,wagon,2008,0,143,a4,150000,gasoline,audi,0


In [31]:
df_car_pure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291342 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             291342 non-null  int64 
 1   VehicleType       291342 non-null  object
 2   RegistrationYear  291342 non-null  int64 
 3   AutoGearbox       291342 non-null  int64 
 4   Power             291342 non-null  int64 
 5   Model             291342 non-null  object
 6   Kilometer         291342 non-null  int64 
 7   FuelType          291342 non-null  object
 8   Brand             291342 non-null  object
 9   NotRepaired       291342 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 24.5+ MB


### Вывод

1. Были загружены данные, выявлены и удалены аномалии в 3-х столбцах: `'RegistrationYear'`,`'Power'`, `'Price'` (было удалено 17,79% данных)

2. Были удалены стоблцы, не влияющие на целевой признак.

3. Заполнены пропуски в столбцах `'VehicleType'`, `'Gearbox'`, `'Model'`, `'FuelType'`, `'NotRepaired'` - 
* в столбцах `'VehicleType'` и `'Model'` нельзя восстановить информацию, поэтому пропуски были заменены на значение 'unknown', 
* столбцы `'Gearbox'` и `'FuelType'` были заполнены самыми частовстречающимися значениями,
* столбец `'NotRepaired'` заполнен значением 'yes', подразумевая, что автомобиль не был ремонтирован.

4. Признаки `'Gearbox'` и `'NotRepaired'` преобразованы в бинарные.

## Обучение моделей

Обучим следующие модели:
1. LinearRegression
2. RandomForestRegressor
3. CatBoostRegressor
4. LGBMRegressor
5. DummyRegressor

### Кодирование признаков и разбиение на обучающую и тестовую выборки

Для обучения моделей необходимо закодировать категориальные признаки и подготовить тестовую выборку.

In [32]:
# Без кодирования
features_original = df_car_pure.drop('Price', axis=1)

# Целевой признак 
target = df_car_pure['Price']

# One Hot Encoding (OHE)
features_ohe = pd.get_dummies(features_original, drop_first=True)

# Ordinal Encoding 
cat_columns = ['VehicleType', 'Model', 'FuelType','Brand']
features_ord = features_original.copy()
for column in cat_columns:
    features_ord[column] = features_ord[column].astype('category') 
    features_ord[column] = features_ord[column].cat.codes

In [33]:
features_original_train, features_original_test, target_train, target_test = train_test_split(
    features_original, target, test_size=0.25, random_state=1)

features_ord_train = features_ord.loc[features_original_train.index, :]
features_ord_test = features_ord.loc[features_original_test.index, :]

features_ohe_train = features_ohe.loc[features_original_train.index, :]
features_ohe_test = features_ohe.loc[features_original_test.index, :]

In [35]:
print(f'Размер тренировочной выборки: {features_original_train.shape}')
print(f'Размер тестовой выборки: {features_original_train.shape}')
print(f'Размер тренировочной выборки, закодированной OE: {features_ord_train.shape}')
print(f'Размер тестовой выборки, закодированной OE: {features_ord_test.shape}')
print(f'Размер тренировочной выборки, закодированной OHE: {features_ohe_train.shape}')
print(f'Размер тестовой выборки, закодированной OHE: {features_ohe_test.shape}')

print(f'Размер целевой тренировочной выборки: {target_train.shape}')
print(f'Размер целевой тестовой выборки: {target_test.shape}')

Размер тренировочной выборки: (218506, 9)
Размер тестовой выборки: (218506, 9)
Размер тренировочной выборки, закодированной OE: (218506, 9)
Размер тестовой выборки, закодированной OE: (72836, 9)
Размер тренировочной выборки, закодированной OHE: (218506, 307)
Размер тестовой выборки, закодированной OHE: (72836, 307)
Размер целевой тренировочной выборки: (218506,)
Размер целевой тестовой выборки: (72836,)


Для разных моделей подойдет разное кодирование:

1. Без кодирования данные (и категориальные, и количественные) будут использованы для модели CatBoost, которая сама кодирует категориальные признаки.
2. Закодированные техникой One Hot Encoding подойдут для линейной регрессии, так как номер модели, закодированный Ordinal Encoding не нужно сравнивать как больше/меньше.
3. Ordinal Encoding подойдет моделям случайного леса и LightGBM, так как моделям, основанным на деревьях решений не нужно сравнивать числа как большие или меньшие.

### Модель LinearRegression

In [ ]:
%%time
# обучим модель
lin_reg = LinearRegression()
parameters_lin_reg = {'fit_intercept':[True, False], 
                      'normalize':[True, False]}

                   
grid_lin_reg = GridSearchCV(lin_reg, parameters_lin_reg, 
                            scoring='neg_root_mean_squared_error', 
                            n_jobs=-1, cv=4, 
                            return_train_score=True)
grid_lin_reg.fit(features_ohe_train, target_train)

In [70]:
# полученное значение rmse
grid_lin_reg.best_score_*(-1)

2612.2948679393353

In [71]:
# посмотрим на лучшее значение 
grid_lin_reg.best_params_

{'fit_intercept': True, 'normalize': True}

In [72]:
grid_lin_reg.cv_results_

{'mean_fit_time': array([8.8494994 , 7.35250026, 7.69174999, 5.85526299]),
 'std_fit_time': array([0.67830823, 0.84645604, 0.5205226 , 0.36836151]),
 'mean_score_time': array([0.20975167, 0.24874973, 0.21374983, 0.15150183]),
 'std_score_time': array([0.00649618, 0.0371101 , 0.04715487, 0.02432265]),
 'param_fit_intercept': masked_array(data=[True, True, False, False],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_normalize': masked_array(data=[True, False, True, False],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'fit_intercept': True, 'normalize': True},
  {'fit_intercept': True, 'normalize': False},
  {'fit_intercept': False, 'normalize': True},
  {'fit_intercept': False, 'normalize': False}],
 'split0_test_score': array([-2617.71401384, -2617.90218506, -2883.87536982, -2883.87536982]),
 'split1_test_score': array([-2608.56050998, -2608.5329437 , -2850

### RandomForestRegressor

In [73]:
%%time
# обучим модель
many_tree = RandomForestRegressor(random_state=1)
parameters_many_tree = {'n_estimators': range(50,300,50),
                        'min_samples_split': range(5,12,3), 
                        'max_depth':range(1,15,5),
                        'min_samples_leaf':range(1,12,5)}

                    
grid_many_tree = GridSearchCV(many_tree, parameters_many_tree, 
                                scoring='neg_root_mean_squared_error', 
                                n_jobs=-1, cv=3, 
                                return_train_score=True)
grid_many_tree.fit(features_ord_train, target_train)

Wall time: 30min 44s


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'max_depth': range(1, 15, 5),
                         'min_samples_leaf': range(1, 12, 5),
                         'min_samples_split': range(5, 12, 3),
                         'n_estimators': range(50, 300, 50)},
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [74]:
# полученное значение rmse
grid_many_tree.best_score_*(-1)

1825.7604861114726

In [75]:
# посмотрим на лучшее значение параметров
grid_many_tree.best_params_

{'max_depth': 11,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 250}

In [2]:
#grid_many_tree.cv_results_

### CatBoostRegressor

In [3]:
# Подбор гиперпараметров

#%%time
#model_cat = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', 
                              #border_count = 32,
                              #learning_rate = 0.1,
                              #l2_leaf_reg = 3,
                              #cat_features = cat_columns,
                              #thread_count = -1,
                              #random_seed=1)

#grid_cat = {'depth': [10, 15, 20]}
        
#grid_search_result_cat = model_cat.grid_search(grid_cat,
                                       #X=features_original_train,
                                       #y=target_train,
                                       #cv=4,
                                       #partition_random_seed=1,
                                       #plot=True)

In [85]:
model_cat.best_score_

{'learn': {'RMSE': 1392.832026024745}}

In [81]:
grid_search_result_cat['params']

{'depth': 10}

### LightGBMRegressor

In [93]:
%%time

model_lgbm = LGBMRegressor(random_state=1) 
grid_lgbm = {'num_leaves':[100, 200, 300], 
             'learning_rate':[0.1],
             'n_estimators':[300, 500, 900]}


grid_search_lgbm = GridSearchCV(model_lgbm, grid_lgbm, 
                                scoring='neg_root_mean_squared_error',
                                n_jobs=-1, cv=3, 
                                return_train_score=True)
grid_search_lgbm.fit(features_ord_train, target_train)

Wall time: 5min 27s


GridSearchCV(cv=3, estimator=LGBMRegressor(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': [0.1],
                         'n_estimators': [300, 500, 600, 900],
                         'num_leaves': [100, 200, 400]},
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [94]:
# полученное значение rmse
grid_search_lgbm.best_score_*(-1)

1599.8111702299348

In [95]:
# посмотрим на лучшее значение параметров
grid_search_lgbm.best_params_

{'learning_rate': 0.1, 'n_estimators': 900, 'num_leaves': 100}

### DummyRegressor

In [100]:
dummy_reg = DummyRegressor(strategy='median')
dummy_reg.fit(features_ord_train, target_train)
dummy_pred = dummy_reg.predict(features_ord_train)
print((mean_squared_error(target_train, dummy_pred))**0.5)

4866.338344170001


### Вывод

Данные были закодированы методами One Hot Encoding и Ordinal Encoding.

Были обучены модели:
1. Линейная регрессия, RMSE = 2612.3, время обучения 28 секунд;
2. Случайный лес, RMSE = 1825.8, время обучения 30 минут 44 секунд;
3. Модель CatBoost, RMSE = 1606.5, время обучения 18 минут 39 секунд;
4. Модель LightGBM, RMSE = 1599.8, время обучения 5 минут 27 секунд.

Также модели были сравнены со случайной, которая показала худший результат RMSE = 4866.3.


Разница во времени обучения связана, в первую очередь, с количеством перебора гиперпараметров с помощью GridSearchCV. Но при этом случайный лес обучался дольше всего (больше всего было перебрано гиперпараметров), но показал результат хуже, чем модели бустинга.

## Анализ моделей

Обучим модели с наилучшими гиперпараметрами и вычислим итоговое значение RMSE.

In [101]:
# Формулу для получения значения RMSE
def rmse (target, predictions):
    return mean_squared_error(target, predictions)** 0.5

### Модель LinearRegression

In [102]:
%%time

lin_reg = LinearRegression(normalize=True, n_jobs=-1)
lin_reg.fit(features_ohe_train, target_train)

C:\Users\ar_ma\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


Wall time: 4.13 s


LinearRegression(n_jobs=-1, normalize=True)

In [103]:
%%time
pred_lin_reg = lin_reg.predict(features_ohe_test)

Wall time: 120 ms


In [105]:
print(f'RMSE на тестовой выборке = {rmse(target_test, pred_lin_reg)}')

RMSE на тестовой выборке = 2627.5867820620347


### RandomForestRegressor

In [113]:
%%time

many_tree = RandomForestRegressor(max_depth=11,
                                  min_samples_leaf=1,
                                  min_samples_split=5,
                                  n_estimators=250, 
                                  n_jobs=-1,random_state=1)
many_tree.fit(features_ord_train, target_train)

Wall time: 16 s


RandomForestRegressor(max_depth=11, min_samples_split=5, n_estimators=250,
                      n_jobs=-1, random_state=1)

In [114]:
%%time
pred_many_tree = many_tree.predict(features_ord_test)

Wall time: 358 ms


In [115]:
print(f'RMSE на тестовой выборке = {rmse(target_test, pred_many_tree)}')

RMSE на тестовой выборке = 1822.5535586615622


### CatBoostRegressor

In [125]:
%%time
model_cat = CatBoostRegressor(loss_function='RMSE', 
                               cat_features = cat_columns,
                               depth=10,
                               border_count=32,
                               l2_leaf_reg=3,
                               learning_rate =0.1,
                               iterations=800,
                               eval_metric='RMSE',
                               thread_count=-1,
                               random_seed=1) 

model_cat.fit(features_original_train, target_train, metric_period=200)  

0:	learn: 4252.8326749	total: 159ms	remaining: 2m 6s
200:	learn: 1585.0673123	total: 27.7s	remaining: 1m 22s
400:	learn: 1509.3968881	total: 55.9s	remaining: 55.6s
600:	learn: 1457.8171139	total: 1m 26s	remaining: 28.5s
799:	learn: 1421.0299476	total: 1m 55s	remaining: 0us
Wall time: 1min 56s


In [126]:
%%time
pred_cat = model_cat.predict(features_original_test)

Wall time: 301 ms


In [127]:
print(f'RMSE на тестовой выборке = {rmse(target_test, pred_cat)}')

RMSE на тестовой выборке = 1610.6533008864772


### LightGBMRegressor

In [128]:
%%time

model_lgbm = LGBMRegressor(num_leaves=100,
                           learning_rate=0.1,
                           n_estimators=900, 
                           n_jobs=-1,
                           random_state=1) 

model_lgbm.fit(features_ord_train, target_train)

Wall time: 4.01 s


LGBMRegressor(n_estimators=900, num_leaves=100, random_state=1)

In [144]:
%%time
pred_lgbm = model_lgbm.predict(features_ord_test)

Wall time: 975 ms


In [130]:
print(f'RMSE на тестовой выборке = {rmse(target_test, pred_lgbm)}')

RMSE на тестовой выборке = 1572.373047585002


### Вывод

Были обучены модели с наилучшими гиперпараметрами, которые были найдены в предыдущем пункте.

In [145]:
df_dict = [[4.13, 120, 2627.59],
            [16, 358, 1822.55],
            [116, 301, 1610.65],
            [4.01, 975, 1572.37]]
model_name = ["Linear Regression", "RandomForestRegressor", "CatBoostRegressor", "LGBMRegressor"]

In [146]:
df_total = pd.DataFrame(data=df_dict, index=model_name, columns=["Время обучения, s", "Время предсказания, ms", "RMSE"])
df_total['RMSE отношение'] = (df_total['RMSE'].min() / df_total['RMSE'])
df_total['Время обучения отношение'] = (df_total['Время обучения, s'].min() / df_total['Время обучения, s'])
df_total['Время предсказания отношение'] = (df_total['Время предсказания, ms'].min() / df_total['Время предсказания, ms'])
df_total['Итого'] = df_total['RMSE отношение']+ df_total['Время обучения отношение'] + df_total['Время предсказания отношение']
df_total

,"Время обучения, s","Время предсказания, ms",RMSE,RMSE отношение,Время обучения отношение,Время предсказания отношение,Итого
Linear Regression,4.13,120,2627.59,0.598408,0.970944,1.000000,2.569352
RandomForestRegressor,16.00,358,1822.55,0.862731,0.250625,0.335196,1.448551
CatBoostRegressor,116.00,301,1610.65,0.976233,0.034569,0.398671,1.409473
LGBMRegressor,4.01,975,1572.37,1.000000,1.000000,0.123077,2.123077


## Вывод

Для выбора ниалучшей моделей, нужно учесть:

* качество предсказания;
* скорость предсказания;
* время обучения.

Модель, которая показала самый лучший результат - LGBMRegressor, также показала минимальное время обучения, но время предсказания самое большое.

RandomForestRegressor недолго обучается и недолго предсказывает, но точность у него ниже.

CatBoostRegressor показал себя чуть хуже LGBMRegressor, но время обучения очень большое.

Linear Regression показала плохую точность.

Очень много зависит от мощности машины, а также случайности. Также важно знать, какая точность нам нужна. 

* LGBMRegressor - если нужна точность.
* RandomForestRegressor - если важно время.